In [6]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul  8 13:32:55 2020

@author: work14
"""

import os
import time
import pyTSL
import datetime
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',200)
pd.options.mode.chained_assignment = None
import pymongo


def conDB(configuration):
    url = 'mongodb://{user}:{password}@{host}:{port}/{db}'.format(**configuration)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[configuration['db']]
    return db

def as_list(x):
    if not (x is None or isinstance(x, list)):
        x = [x]
    return x

def read_mongo(config_db, name, skey=None, start=None, end=None, interval=None, col=None, return_sdi=True):
    # Regularize 'skey' and 'interval'
    skey = as_list(skey)
    interval = as_list(interval)

    # Get collection
    db = conDB(config_db)
    collection = db[name]

    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start is not None:
        if end is not None:
            query['date'] = {'$gte': start, '$lte': end}
        else:
            query['date'] = {'$gte': start}
    elif end is not None:
        query['date'] = {'$lte': end}

    # Load data
    cur = collection.find(query, prj).batch_size(1000000)
    if cur is not None:
        df = pd.DataFrame.from_records(cur)
    else:
        df = pd.DataFrame()
    return df

def to_mongo(config_db, name, df):
    # Get collection
    db = conDB(config_db)
    collection = db[name]

    # Insert dataframe
    df = df.to_dict('records')
    collection.insert_many(df)
    

def getTradeDate(begT, endT):
    tsstr = """    begt := %s;
                   endt := %s;
                   begt_:=datetoint(begt);
                   endt_:=datetoint(endt);
                   dayArr:=sselect inttodate(['截止日']) from infotable 753 of 'SH000001'
                               where ['截止日']>=begt_
                                     and ['截止日']<=endt_
                                     and ['是否交易日']=1
                                     order by ['截止日'] end;
                   if not istable(dayArr) then endt1:=endt;
                   else endt1:=dayArr[0];
                   hisArr:=MarketTradeDayQk(begt,endt1);
                   dateDf := hisArr union2 dayArr;
                   dateDf := select [0] as 'date' from `dateDf end;
                   dateDf[:]['date'] := datetostr(dateDf[:]['date']);
                   return dateDf; """%(begT + 'T', endT + 'T')
    dateDf = pd.DataFrame(c.exec(tsstr).value())    
    return dateDf


def download_daily_TR(begDate, endDate):
    TRBegDate = datetime.datetime.strptime(begDate,"%Y-%m-%d").strftime('%Y%m%d')
    TREndDate = datetime.datetime.strptime(endDate,"%Y-%m-%d").strftime('%Y%m%d')
#    startTm =datetime.datetime.now()
    tsstr="""
            BegT :={};
            EndT :={} + 0.99;
            setSysParam(pn_date(),EndT);
            setSysParam(pn_cycle(),cy_day());
            dateList:=MarketTradeDayQk(BegT,EndT);
            re:=array();
            k:=0;
            for i:=0 to length(dateList)-1 do
            begin
        
               vEndt:=dateList[i]; //use vEndt as cursor
               //StockArr:=getabkbydate('A股',vEndt);
               //StockArr:= array('SH000016','SH000300','SH000852','SH000905','SH000985');
               StockArr:= array('SH000016','SH000300','SH000852','SH000905','SH000985') union2 getabkbydate('A股',vEndt);
               setsysparam(pn_date(),vEndt);
        
               // get index weight
               GetBkWeightByDate('SH000300',vEndt,weightIF);
               weightIF := weightIF[:,array("截止日","代码","比例(%)")];
               GetBkWeightByDate('SH000905',vEndt,weightIC);
               weightIC := weightIC[:,array("截止日","代码","比例(%)")];
               GetBkWeightByDate('SH000852',vEndt,weightCSI1000);
               weightCSI1000 := weightCSI1000[:,array("截止日","代码","比例(%)")];
               GetBkWeightByDate('SH000985',vEndt,weightCSIRest);
               weightCSIRest := weightCSIRest[:,array("截止日","代码","比例(%)")];
               
               for j:=0 to length(StockArr)-1 do
               begin
                 setsysparam(pn_stock(),StockArr[j]);
        
                 // base info
                 re[k]['skey']:=StockArr[j];
                 re[k]['date']:=datetostr(vEndt);
                 re[k]['name']:=StockNameEndT(StockArr[j], vEndt);
                 re[k]['trade_status']:=istradeday(vEndt);
                 re[k]['listed_days'] := tradedays(StockFirstDay(StockArr[j]),vEndt); //stock total trading days
                                  //市场交易天数：MarketTradeDays(BegT,EndT);自然天数：EndT-BegT
                 // price info
                 re[k]['open']:=open();
                 re[k]['high']:=high();
                 re[k]['low']:=low();
                 re[k]['close']:=close();
                 re[k]['closeL1']:=specdate(close(),vEndt-1);//昨天的价格（考虑到有停牌） 若想更改为上一个交易日的价格就改为ref(close(),1)
                 re[k]['yclose']:=Sys_PrevClose();
                 re[k]['ztClose']:=StockZtClose(vEndt);
                 re[k]['dtClose']:=StockDtClose(vEndt);
                 re[k]['dayReturn']:=StockZf3()/100;
        
                 // volume and amount
                 re[k]['volume']:=vol();
                 re[k]['amount']:=amount();
                 re[k]['buy_volume']:=StockBuyVol3();
                 re[k]['sell_volume']:=StockSellVol3();
                 re[k]['TORate']:=StockHsl3()/100;
                 
                 // zt, dt , ST
                 re[k]['allZT']:=StockIsZt2(vEndt);
                 re[k]['isZT']:=StockIsZt(vEndt);
                 re[k]['hasZT']:=StockIsCjZt(vEndt);
                 re[k]['allDT']:=StockIsDt2(vEndt);
                 re[k]['isDT']:=StockIsDt(vEndt);
                 re[k]['hasDT']:=StockIsCjDt(vEndt);
                 re[k]['isST']:=IsST_3(vEndt);
                 
                 // other Info
                 re[k]['VWAP']:=StockVWAP4(vEndt,0,0.99);
                 re[k]['TWAP']:=StockTWAP4(vEndt,0,0.99);
                 re[k]['marketValue']:=StockMarketValue3()*10000;
                 re[k]['marketShares']:=StockNegotiableShares3();
                 re[k]['totalShares']:=StockTotalShares3();
        
                 // industry info
                 re[k]['SW1_name']:=StockSWIndustryNameLv1()?:base(10029);
                 re[k]['SW1_code']:=StockSWIndustryIdLv1()?:base(10035);
                 re[k]['SW2_code']:=StockSWIndustryIdLv2()?:base(10036);
                 re[k]['SW3_code']:=StockSWIndustryIdLv3()?:base(10037);
                 
                 // index info
                 re[k]['index_name'] := 'None';
                 re[k]['index_weight'] := 0.0;
                 if StockArr[j] in weightIF then
                    begin
                    re[k]['index_name'] := 'IF';
                    re[k]['index_weight'] := vselect ['比例(%)'] from weightIF
                    where ['代码'] = StockArr[j] end;
                    end
                 else if StockArr[j] in weightIC then
                    begin
                    re[k]['index_name'] := 'IC';
                    re[k]['index_weight'] := vselect ['比例(%)'] from weightIC
                    where  ['代码'] = StockArr[j] end;
                    end
                 else if StockArr[j] in weightCSI1000 then
                    begin
                    re[k]['index_name'] := 'CSI1000';
                    re[k]['index_weight'] := vselect ['比例(%)'] from weightCSI1000
                    where  ['代码'] = StockArr[j] end;
                    end
                 else if StockArr[j] in weightCSIRest then
                    begin
                    re[k]['index_name'] := 'CSIRest';
                    re[k]['index_weight'] := vselect ['比例(%)'] from weightCSIRest
                    where  ['代码'] = StockArr[j] end;
                    end
                 
                 k++;
               end
            end
            return re;
             """.format(TRBegDate + 'T', TREndDate + 'T')
    df_day = pd.DataFrame(c.exec(tsstr).value())  
    assert len(df_day) > 0
    assert df_day['index_name'].isin(['IF','IC','CSI1000','CSIRest','None']).all()
    ## set date and insert time
    df_day['date'] = df_day['date'].map(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").strftime('%Y%m%d'))
#    timeNow = datetime.datetime.now()
#    df_day['time'] = int(str(timeNow.hour).zfill(2) + str(timeNow.minute).zfill(2)  + str(timeNow.second).zfill(2) + str(timeNow.microsecond).zfill(6))
    df_day['time'] = 180000000000

    ## define exchange and skey
    df_day['exchange'] = df_day['skey'].map(lambda x:x[:2])
    assert df_day['exchange'].isin(['SH','SZ']).all()
    df_day['exchange'] = np.where(df_day['exchange'] == 'SH', 'SSE', 'SZSE')
    assert df_day['exchange'].isin(['SSE','SZSE']).all()
    df_day['skey'] = df_day['skey'].map(lambda x:x[2:])
    df_day['skey'] = np.where(df_day['exchange'] == 'SSE', '1' + df_day['skey'], '2' + df_day['skey'])
    
    ## set data type
    for col in ['trade_status','allZT','hasZT','isZT','allDT','hasDT','isDT','isST']:
        df_day[col] = df_day[col].astype('int8')
    for col in ['skey','date','listed_days']:
        df_day[col] = df_day[col].astype('int32')
    for col in ['time','volume']:
        df_day[col] = df_day[col].astype('int64')
    for col in ['open','high','low','close','closeL1','yclose','ztClose','dtClose','dayReturn','amount','buy_volume',
                'buy_volume','TORate','VWAP','TWAP','marketValue','marketShares','totalShares','index_weight']:
        df_day[col] = df_day[col].astype('float64')
    for col in ['name','SW1_name','SW1_code','SW2_code','SW3_code','index_name','exchange']:
        df_day[col] = df_day[col].astype('str')
        
    df_day = df_day[['skey','date','time','name','trade_status','listed_days','open','high','low','close','closeL1',
                         'yclose','ztClose','dtClose','dayReturn','volume','amount','buy_volume','sell_volume','TORate',
                         'allZT','hasZT','isZT','allDT','hasDT','isDT','isST','VWAP','TWAP','marketValue','marketShares',
                         'totalShares','SW1_name','SW1_code','SW2_code','SW3_code','index_name','index_weight','exchange']].reset_index(drop = True)
    print('        Number of Stocks Downloaded: ', df_day.skey.nunique())
#    print(TRBegDate, ' download time: ',datetime.datetime.now() - startTm)
    
    return df_day



config = {
    'user': "zhenyuy",
    'password': "bnONBrzSMGoE",
    'host': '192.168.10.178',
    'port': '27017',
    'db': 'com_md_eq_cn'
} 
if __name__ == '__main__':  
    
    ## TR login in 
    c = pyTSL.Client("jqtz", "+7.1q2w3e", "tsl.tinysoft.com", 443) ##pyTSL.Client(TR.ini)
    c.login()
    assert c.login() == 1
    
    try: 
        print('\n', '......BEGIN DOWNLOADING TR......')
        dateList = sorted(getTradeDate('20201111', '20201113').date.unique())        
#         dateList = [datetime.datetime.today().strftime('%Y-%m-%d')]
        for date in dateList[::-1]:
             
            """
            STEP 1: download daily data from TR
            """   
            startTm = time.time()
#             savePath = os.path.join('your path!!!','TRData','MongoDB','daily', date[:4])
#             os.makedirs(savePath, exist_ok = True)
            df_day = download_daily_TR(date, date)
#             df_day.to_csv(os.path.join(savePath, 'stock_' + date + '.csv' ), index = False, encoding = 'gbk')
            print('... download TR %s: Takes %d sec' % (date, time.time() - startTm)) 

            """
            STEP 2: upload daily data to DataBase
            """   
            startTm = time.time()
#            df_day = pd.read_csv(os.path.join(savePath, 'stock_' + date + '.csv' ), encoding = 'gbk')
            df_old  = read_mongo(config, 'mdbar1d_tr', skey = 1600000, start = 20200101, col = ['skey','date','trade_status'])
            assert len(df_old[df_old.duplicated()]) == 0
            assert df_day.date.min() > df_old.date.max()
            assert int(sorted(getTradeDate('20200101', date.replace('-','')).date.unique())[-2].replace('-','')) == df_old.date.max()
            for col in ['trade_status','allZT','hasZT','isZT','allDT','hasDT','isDT','isST']:
                df_day[col] = df_day[col].astype('int8')
            for col in ['skey','date','listed_days']:
                df_day[col] = df_day[col].astype('int32')
            for col in ['time','volume']:
                df_day[col] = df_day[col].astype('int64')
            for col in ['open','high','low','close','closeL1','yclose','ztClose','dtClose','dayReturn','amount','buy_volume',
                        'buy_volume','TORate','VWAP','TWAP','marketValue','marketShares','totalShares','index_weight']:
                df_day[col] = df_day[col].astype('float64')
            for col in ['name','SW1_name','SW1_code','SW2_code','SW3_code','index_name','exchange']:
                df_day[col] = df_day[col].astype('str')
            print(np.sort(df_day['skey']))        
#             to_mongo(config, 'mdbar1d_tr', df_day)
            print('...   upload DB %s: Takes %d sec' % (date, time.time() - startTm))      
        print('......FINISH UPLOADING DB......', '\n')
            
    except Exception:
        print(Exception)
        print('...DOWNLOAD ERROR, PlEASE CHECK MANUALLY!!!')
    
    finally:
        ## TR log out
        c.logout()   





 ......BEGIN DOWNLOADING TR......
        Number of Stocks Downloaded:  4068
... download TR 2020-11-13: Takes 81 sec
<class 'Exception'>
...DOWNLOAD ERROR, PlEASE CHECK MANUALLY!!!
